In [3]:
#import needed libraries
import warnings
warnings.filterwarnings('ignore')
from sqlalchemy import create_engine
import pyodbc
import pandas as pd
import os
import psycopg2

host= "localhost:5432"
user= "postgres"
password= "zxc"
nombre_db= "DB_Projects"

driver = "{SQL Server}"
server = "DESKTOP-7MLFO65"
database = "PortfolioProjects"


In [4]:
#extract data from sql server
def extract():
    try:
        src_conn = pyodbc.connect('DRIVER=' + driver + ';SERVER=' + server + ';DATABASE=' + database + ';Trust_Connection=yes')
        src_cursor = src_conn.cursor()
        # execute query
        src_cursor.execute(""" select  t.name as table_name
        from sys.tables t where t.name in ('CovidVaccinations','CovidDeaths') """)
        src_tables = src_cursor.fetchall()
        for tbl in src_tables:
            #query and load save data to dataframe
            df = pd.read_sql_query(f'select * FROM {tbl[0]}', src_conn)
            load(df, tbl[0])
            print
    except Exception as e:
        print("Data extract error: " + str(e))
    finally:
        src_conn.close()

#load data to postgres
def load(df, tbl):
    try:
        rows_imported = 0
        url = f"postgresql://{user}:{password}@{host}/{nombre_db}"
        engine =create_engine(url)
        print(f'importing rows {rows_imported} to {rows_imported + len(df)}... for table {tbl}')
        # save df to postgres
        df.to_sql(f'stg_{tbl}', engine, if_exists='replace', index=False)
        rows_imported += len(df)
        # add elapsed time to final print out
        print("Data imported successful")
    except Exception as e:
        print("Data load error: " + str(e))

try:
    #call extract function
    extract()
except Exception as e:
    print("Error while extracting data: " + str(e))

importing rows 0 to 85171... for table CovidDeaths
Data imported successful
importing rows 0 to 85171... for table CovidVaccinations
Data imported successful
